In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
from liualgotrader.analytics import analysis 
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import sys
import pytz

In [2]:
day_to_analyze = datetime.strptime("2020-09-25", "%Y-%m-%d")

In [3]:
trades = analysis.load_trades(day_to_analyze)

In [4]:
algo_runs = analysis.load_runs(day_to_analyze)

In [ ]:
symbol_name = []
counts = []
revenues = []

how_was_my_day = pd.DataFrame()
how_was_my_day["symbol"] = trades.symbol.unique()
how_was_my_day["revenues"] = how_was_my_day["symbol"].apply(analysis.calc_revenue)
how_was_my_day["count"] = how_was_my_day["symbol"].apply(count_trades)

In [ ]:
minute_history = {}
for batch_id, count in batch.items():
    for run_id in batch[batch_id]:
        symbols = trades.loc[trades["algo_run_id"] == run_id]["symbol"].value_counts()
        for symbol, count in symbols.items():
            if symbol not in minute_history:
                minute_history[symbol] = api.polygon.historic_agg_v2(
                    symbol,
                    1,
                    "minute",
                    _from=str(
                        (
                            datetime.strptime(start_day_to_analyze, "%Y-%m-%d")
                            - timedelta(days=10)
                        ).date()
                    ),
                    to=str(
                        (
                            datetime.strptime(end_day_to_analyze, "%Y-%m-%d")
                            + timedelta(days=1)
                        ).date()
                    ),
                ).df

In [ ]:
def grouper(iterable):
    prev = None
    group = []
    for item in iterable:

        if not prev or -0.02 <= float(item - prev) / prev <= 0.02:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group


def find_resistance(current_value, minute_history, now):
    """calculate next resistance"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(pytz.utc.localize(now).astimezone(est))

    minute_history_index = minute_history["close"].index.get_loc(now, method="nearest")
    back_time = pd.Timestamp(now.to_pydatetime() - timedelta(days=4))
    back_time_index = minute_history["close"].index.get_loc(back_time, method="nearest")
    for back_track_min in range(minute_history_index + 1 - 120, back_time_index, -5):

        series = (
            minute_history["close"][back_track_min : minute_history_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("5min")
            .max()
        ).dropna()

        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] >= current_value]
            )
            if len(local_maximas) > 0:
                return local_maximas
            continue

            clusters = dict(enumerate(grouper(local_maximas), 1))
            resistances = []
            for key, cluster in clusters.items():
                if len(cluster) > 1:
                    resistances.append(round(sum(cluster) / len(cluster), 2))
            resistances = sorted(resistances)

            if len(resistances) > 0:
                return resistances

    return None


def find_support(current_value, minute_history, now):
    """calculate support"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(pytz.utc.localize(now).astimezone(est))
    minute_history_index = minute_history["close"].index.get_loc(now, method="nearest")
    back_time = pd.Timestamp(now.to_pydatetime() - timedelta(days=4))
    back_time_index = minute_history["close"].index.get_loc(back_time, method="nearest")
    for back_track_min in range(
        minute_history_index + 1 - 120, back_time_index, -5
    ):  # len(minute_history[:minute_history_index+1].index),
        series = (
            minute_history["close"][back_track_min : minute_history_index + 1]
            .resample("5min")
            .min()
        ).dropna()
        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] <= 0) & (diff[1:] > 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] <= current_value]
            )
            if len(local_maximas) > 0:
                return local_maximas

    return None

In [ ]:
est = pytz.timezone("US/Eastern")
position = {}
for symbol in minute_history:
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = symbol_df["tstamp"].min().to_pydatetime()
    start_date = start_date.replace(hour=9, minute=30)
    cool_down_date = start_date + timedelta(minutes=5)
    try:
        minute_history_index = minute_history[symbol]["close"].index.get_loc(
            start_date, method="nearest"
        )
    except Exception as e:
        print(f"Error for {symbol}: {e}")
        continue

    cool_minute_history_index = minute_history[symbol]["close"].index.get_loc(
        cool_down_date, met
        
        hod="nearest"
    )
    open_price = minute_history[symbol]["close"][cool_minute_history_index]
    plt.plot(
        minute_history[symbol]["close"][minute_history_index:].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.xticks(rotation=45)

    delta = 0
    profit = 0

    operations = []
    deltas = []
    profits = []
    times = []
    prices = []
    qtys = []
    indicators = []
    target_price = []
    stop_price = []
    resistances = []
    supports = []
    daily_change = []
    precent_vwap = []
    position[symbol] = 0
    for index, row in symbol_df.iterrows():
        resistance = None
        support = None
        if not position[symbol]:
            try:
                now = int(row["client_time"])
                continue
            except Exception:
                pass

            resistance = find_resistance(
                row["price"], minute_history[symbol], row["tstamp"]
            )
            if resistance:
                for r in resistance:
                    plt.axhline(y=r, color="r")
            support = find_support(row["price"], minute_history[symbol], row["tstamp"])
            if support:
                for s in support:
                    plt.axhline(y=s, color="g")

        if position[symbol] >= 0 and row["operation"] == 'buy':
            delta = -row["price"] * row["qty"]
            position[symbol] += row["qty"]
        elif position[symbol] <= 0 and row["operation"] == 'sell':
            delta = row["price"] * row["qty"] 
            position[symbol] -= row["qty"]
        elif position[symbol] > 0 and row["operation"] == 'sell':
            delta = row["price"] * row["qty"]
            position[symbol] -= row["qty"]
        elif position[symbol] < 0 and row["operation"] == 'buy':
            delta = -row["price"] * row["qty"]
            position[symbol] += row["qty"]
            
        profit += delta
        plt.scatter(
            row["tstamp"].to_pydatetime(),
            row["price"],
            c="g"
            if row["operation"] == "buy" 
            else "r",
            s=100,
        )
        deltas.append(round(delta, 2))
        profits.append(round(profit, 2))
        operations.append(row["operation"])
        times.append(pytz.utc.localize(pd.to_datetime(row["tstamp"])).astimezone(est))
        prices.append(row["price"])
        qtys.append(row["qty"])
        indicators.append(row["indicators"])
        target_price.append(row["target_price"])
        stop_price.append(row["stop_price"])
        resistances.append(resistance if resistance else None)
        supports.append(support)
        daily_change.append(
            f"{round(100.0 * (row['price'] - open_price) / open_price, 2)}%"
        )
        precent_vwap.append(
            f"{round(100.0 * (row['indicators']['avg'] - open_price) / open_price, 2)}%"
            if "avg" in row["indicators"]
            else ""
        )

    d = {
        "profit": profits,
        "trade": deltas,
        "operation": operations,
        "at": times,
        "price": prices,
        "support": supports,
        "resistance": resistances,
        "qty": qtys,
        "daily change": daily_change,
        "vwap": precent_vwap,
        "indicators": indicators,
        "target price": target_price,
        "stop price": stop_price,
    }
    print(f"{symbol} analysis with profit {round(profit, 2)}")
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()